In [8]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import StringType
from snowflake.snowpark.types import IntegerType
from config import connection_parameters


In [9]:
#Definir sesión y objetos Snowflake
sesion = Session.builder.configs(connection_parameters).create()
if sesion != None:
    print("Connection established!")
    sesion.use_database('inegi')
    print(sesion.sql("select current_warehouse(), current_database(), current_role()").collect()) 
else:
    print("Connection error!")

Conectado
[Row(CURRENT_WAREHOUSE()='INEGI_WH', CURRENT_DATABASE()='INEGI', CURRENT_ROLE()='INEGI_ROLE')]


In [10]:
#Create INEGI_DATA view 
query = "create or replace  view INEGI_DATA as select " + \
 "v:ENTIDAD::int as entidad," + \
 "v:MUN::int as municipio," + \
 "v:NOM_MUN::string as nom_municipio," + \
 "v:LOC::string as localidad," + \
 "v:NOM_LOC::string as nom_localidad," + \
 "v:LONGITUD::float as longitud," + \
 "v:LATITUD::float as latitud," + \
 "v:ALTITUD::int as altitud," + \
 "v:POBTOT::int as pob_total," + \
 "v:POBFEM::int as pob_fem," + \
 "v:POBMAS::int as pob_masc," + \
 "v:PCON_DISC::int as pob_discapacidad," + \
 "v:GRAPROES::int as pob_escolaridad," + \
 "v:GRAPROES_F::int as pob_esco_fem," + \
 "v:GRAPROES_M::int as pob_esco_masc," + \
 "v:PSINDER::int as pob_sssalud," + \
 "v:PDER_SS::int as pob_cssalud," + \
 "v:VIVTOT::int as total_vivienda," + \
 "v:TVIVHAB::int total_habitada," + \
 "v:VPH_INTER::int as hab_internet " + \
 "from INEGI_RAW;"
sesion.sql(query).collect()



[Row(status='View INEGI_DATA successfully created.')]

In [11]:
#UDF Declare
entidad_udf = sesion.udf.register_from_file(file_path='@inegi/entidad.py',func_name='nom_entity',return_type=StringType(),input_types=[IntegerType()],is_permanent=True, name="nom_entity",stage_location="@inegi")

In [12]:
#View with totals per entity applying UDF to convert entity number x entity name.
viewquery = "CREATE OR REPLACE VIEW INEGI_MAPA AS " + \
" WITH poblacion_lat AS ( SELECT latitud AS latitude, longitud AS longitude, nom_entity(entidad) as nom_entity," + \
" ROW_NUMBER() OVER(PARTITION BY nom_entity ORDER BY nom_entity DESC) AS row_number FROM INEGI_DATA)," + \
" poblacion_t AS ( SELECT sum(pob_total) AS poblacion_total,nom_entity(entidad) AS nom_entity FROM INEGI_DATA " + \
" GROUP BY entidad ORDER BY poblacion_total DESC )" + \
" SELECT pl.nom_entity,pt.poblacion_total AS total_population, pl.latitude, pl.longitude" + \
" FROM poblacion_lat pl LEFT JOIN poblacion_t pt ON pl.nom_entity = pt.nom_entity" + \
" WHERE row_number = 1;"
sesion.sql(viewquery).collect()

[Row(status='View INEGI_MAPA successfully created.')]

In [13]:
#Data validation with Dataframe
df_entidad = sesion.table("INEGI_MAPA")
df_entidad.show()

----------------------------------------------------------------------------------------
|"NOM_ENTITY"          |"TOTAL_POPULATION"  |"LATITUDE"          |"LONGITUDE"          |
----------------------------------------------------------------------------------------
|TLAXCALA              |1342977             |19.605128611111112  |-98.15592555555556   |
|SINALOA               |3026943             |23.55150777777778   |-106.24352694444444  |
|MEXICO                |16992418            |19.416123055555556  |-98.99091277777778   |
|OAXACA                |4132148             |16.382993333333335  |-95.00850444444444   |
|PUEBLA                |6583278             |18.643517499999998  |-97.84966444444444   |
|COAHUILA DE ZARAGOZA  |3146771             |27.595691666666667  |-103.58616083333332  |
|TABASCO               |2402598             |17.951661666666666  |-92.91535027777779   |
|QUERETARO DE ARTEAGA  |2368467             |21.21680222222222   |-99.47291083333333   |
|BAJA CALIFORNIA SUR 

In [14]:
sesion.close()